# Dataloaders to load partial data

In [1]:
import dask.dataframe as dd
import os
import time

In [8]:
data_parquet = os.path.join('data','train_series.parquet')
data: dd.DataFrame = None
grouped_data = None

def init():
    print(f"Reading {data_parquet}")
    t=time.time()
    # data: dd.DataFrame = dd.read_parquet(data_parquet, columns=['step', 'series_id', 'anglez', 'enmo'], categories=['series_id'])
    global data, grouped_data
    data = dd.read_parquet(data_parquet)#, columns=['step', 'series_id', 'anglez', 'enmo'], categories=['series_id'])
    grouped_data = data.groupby('series_id')
    print(f"Pseudo read and grouped {len(data)} rows from {data_parquet} in {time.time()-t:.3f} seconds")

In [9]:
init()

Reading data\train_series.parquet
Pseudo read and grouped 127946340 rows from data\train_series.parquet in 21.135 seconds


In [10]:
data

,series_id,step,timestamp,anglez,enmo
npartitions=1,,,,,
,object,uint32,object,float32,float32
,...,...,...,...,...


In [11]:
grouped_data

In [21]:
def all_series_ids():
    if hasattr(all_series_ids, 'sids'): return all_series_ids.sids
    all_series_ids.sids = data['series_id'].unique().compute()
    return all_series_ids.sids

In [22]:
all_series_ids()

0      038441c925bb
1      03d92c9f6f8a
2      0402a003dae9
3      04f547b8017d
4      05e1944c3818
           ...     
272    fa149c3c4bde
273    fb223ed2278c
274    fbf33b1a2c10
275    fcca183903b7
276    fe90110788d2
Name: series_id, Length: 277, dtype: object

In [23]:
import pandas as pd

def acc_data_for_child(sid):
    dir_path = os.path.join('data', 'parsed')
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)
    file_path = os.path.join(dir_path, f'{sid}.parquet')
    path_clean = os.path.join(file_path, '.clean.txt')
    if os.path.exists(path_clean):
        return pd.read_parquet(file_path)
    dd.to_parquet(grouped_data.get_group(sid), file_path, write_index=False)
    with(open(path_clean, 'w')) as f:
        f.write('')
    return pd.read_parquet(file_path)



In [24]:
c1 = acc_data_for_child('038441c925bb')

c:\Users\rahul\anaconda3\Lib\site-packages\pyarrow\pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):
c:\Users\rahul\anaconda3\Lib\site-packages\pyarrow\pandas_compat.py:456: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


In [25]:
c1

,series_id,step,timestamp,anglez,enmo
0,038441c925bb,0,2018-08-14T15:30:00-0400,2.636700,0.0217
1,038441c925bb,1,2018-08-14T15:30:05-0400,2.636800,0.0215
2,038441c925bb,2,2018-08-14T15:30:10-0400,2.637000,0.0216
3,038441c925bb,3,2018-08-14T15:30:15-0400,2.636800,0.0213
4,038441c925bb,4,2018-08-14T15:30:20-0400,2.636800,0.0215
...,...,...,...,...,...
389875,038441c925bb,389875,2018-09-06T04:59:35-0400,-27.373899,0.0110
389876,038441c925bb,389876,2018-09-06T04:59:40-0400,-27.493799,0.0110
389877,038441c925bb,389877,2018-09-06T04:59:45-0400,-27.533701,0.0111
389878,038441c925bb,389878,2018-09-06T04:59:50-0400,-28.003599,0.0111


In [26]:
c1 = acc_data_for_child('038441c925bb')

In [27]:
c1

,series_id,step,timestamp,anglez,enmo
0,038441c925bb,0,2018-08-14T15:30:00-0400,2.636700,0.0217
1,038441c925bb,1,2018-08-14T15:30:05-0400,2.636800,0.0215
2,038441c925bb,2,2018-08-14T15:30:10-0400,2.637000,0.0216
3,038441c925bb,3,2018-08-14T15:30:15-0400,2.636800,0.0213
4,038441c925bb,4,2018-08-14T15:30:20-0400,2.636800,0.0215
...,...,...,...,...,...
389875,038441c925bb,389875,2018-09-06T04:59:35-0400,-27.373899,0.0110
389876,038441c925bb,389876,2018-09-06T04:59:40-0400,-27.493799,0.0110
389877,038441c925bb,389877,2018-09-06T04:59:45-0400,-27.533701,0.0111
389878,038441c925bb,389878,2018-09-06T04:59:50-0400,-28.003599,0.0111
